バージョン下げると映像がグチャグチャ

バージョンそのままだとエラーでる

In [ ]:
!pip install scipy==1.2.0
!pip install flask-ngrok flask==0.12.2

In [1]:
!pip install flask-ngrok flask==0.12.2

     |████████████████████████████████| 92kB 5.9MB/s 
  Found existing installation: Flask 1.1.2
    Uninstalling Flask-1.1.2:
      Successfully uninstalled Flask-1.1.2


In [2]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 249, done.
remote: Total 249 (delta 0), reused 0 (delta 0), pack-reused 249
Receiving objects: 100% (249/249), 72.12 MiB | 18.84 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [3]:
cd /content/first-order-model

/content/first-order-model


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
#Writing to html file on colab

html = """

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8" />
</head>
<body>

<form action="/post" method="post" enctype="multipart/form-data">
    <p><input type="file" name="uploadFile"/></p>
    <p><input type="submit" value="send"/></p>
</form>

{% if ulr_Image %}

    <p>Image</p>
    <p><img src="ulr_Image"></p>
    <p><<source src=" driving_video"></p>
    <p>Result</p>   
    <p><<source src="predictions"></p>

{% endif %}

</body>
</html>

"""

#Saving html file
if not os.path.isdir( "templates"):
    os.makedirs( "templates")
with open("templates/index.html", mode='w') as f:
    f.write(html)

#Making tmp folder to use temporarily stored
if not os.path.isdir("tmp"):
    os.makedirs("tmp")

In [18]:
def imageResize(img):
    img = resize(img, (256, 256))[..., :3]

    return img

In [19]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
from skimage import io
import warnings
warnings.filterwarnings("ignore")

# source_image = imageio.imread('/content/drive/My Drive/FOMM/cou1.jpg')
reader = imageio.get_reader('/content/drive/My Drive/FOMM/04.mp4')


#Resize image and video to 256x256

# source_image = resize(source_image, (256, 256))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]


def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

first order

In [32]:
from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte
from flask import Flask, render_template, send_from_directory, request, redirect, url_for
from flask_ngrok import run_with_ngrok
from keras.models import load_model
from skimage import io
import imageio
import numpy as np
import os
from IPython.display import HTML

# ALLOWED_EXTENSIONS = set(['png', 'jpg'])

# モデル読み込み
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                        checkpoint_path='/content/drive/My Drive/FOMM/vox-cpk.pth.tar')


app = Flask(__name__)
run_with_ngrok(app)

# # 画像拡張子判定
# def allwed_file(filename):
#     return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# index表示
@app.route("/")
def index():
    return render_template("index.html")

# メイン処理
@app.route('/post', methods=['POST'])
def post():
    try:
        uploadFile = request.files['uploadFile']
    except:
        return redirect(url_for('index'))

    uploadFile.save("tmp/" + uploadFile.filename)

    # img = imageio.imread("tmp/" + uploadFile.filename)
    img = io.imread("tmp/" + uploadFile.filename)

    # リサイズ
    source_image = imageResize(img)
    


    fileName = str(np.random.randint(1000)) + ".jpg"
    io.imsave("tmp/" + fileName, source_image)
    ulr_Image = "tmp/" + fileName,source_image


    # 実行
    predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

    # io.imsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
    imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)

    video = "/content/generated.mp4"

    # dis = display(source_image, driving_video, predictions)

    return render_template( "index.html",
                           ulr_Image = ulr_Image,
                           source_image = source_image, 
                           driving_video = driving_video,
                           predictions = video)
    

if __name__ == '__main__':
    app.run()


# #save resulting video
# imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://8f4f32641076.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [27/Oct/2020 06:02:54] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Oct/2020 06:02:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2020 06:02:55] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/Oct/2020 06:02:55] "GET /favicon.ico HTTP/1.1" 404 -
100%|██████████| 211/211 [00:04<00:00, 45.70it/s]
127.0.0.1 - - [27/Oct/2020 06:03:04] "POST /post HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Oct/2020 06:03:04] "POST /post HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2020 06:03:04] "GET /ulr_Image HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/Oct/2020 06:03:04] "GET /ulr_Image HTTP/1.1" 404 -
